In [1]:
import pandas as pd
import numpy as np
from functions import *
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

remove_fight_island = False

In [2]:
#Load models
#REMINDER: We are going to need to use 'eval' to get the models usable
with open('../data/models.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
print(len(models))


5


In [3]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20',
  'model_10_28_20'],
 ['LogisticRegression(max_iter=50000, random_state=75)',
  "DecisionTreeClassifier(criterion='entropy', max_depth=10, max_leaf_nodes=336,\n                       min_samples_leaf=0.01, random_state=75,\n                       splitter='random')",
  "RandomForestClassifier(criterion='entropy', max_depth=83, max_leaf_nodes=20,\n                       min_samples_leaf=0.009999000000000001, n_estimators=39,\n                       n_jobs=-1, random_state=75)",
  "GradientBoostingClassifier(max_depth=17, max_features='sqrt',\n                           max_leaf_nodes=316, min_samples_leaf=0.1,\n                           n_estimators=195, random_state=75)",
  'GaussianNB()',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.

In [4]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")

In [5]:
#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.
mask = df['Winner'] == 'Red'
df['label'][mask] = 0
mask = df['Winner'] == 'Blue'
df['label'][mask] = 1

#df["Winner"] = df["Winner"].astype('category')
df = df[(df['Winner'] == 'Blue') | (df['Winner'] == 'Red')]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

#Let's fix the date
df['date'] = pd.to_datetime(df['date'])

#Create a label df:
label_df = df['label']

#Let's create an odds df too:
odds_df = df[['R_odds', 'B_odds']]

#Split the test set.  We are always(?) going to use the last 200 matches as the test set, so we don't want those around
#as we pick models

df_train = df[250:]
odds_train = odds_df[250:]
label_train = label_df[250:]

df_test = df[:250]
odds_test = odds_df[:250]
label_test = label_df[:250]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

250
250
250
4509
4509
4509


<ipython-input-5-d6c2a1ffdd58>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 0
<ipython-input-5-d6c2a1ffdd58>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = 1


In [6]:
if remove_fight_island:
    ##Let's remove the Fight island contests and see how that affects score
    df_test_no_fight_island = df_test[(df_test['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_train_no_fight_island = df_train[(df_train['location'] != 'Abu Dhabi, Abu Dhabi, United Arab Emirates')]
    df_test = df_test_no_fight_island
    df_train = df_train_no_fight_island



print(len(df_test))
print(len(df_train))


250
4509


In [7]:
#Set a value for the nulls in the ranks

weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [8]:
len(models[0])

15

In [9]:
score_list = []
for model_num in range(len(models[0])):
    test_model_name = models[0][model_num]
    test_model = eval(models[1][model_num])
    test_model_features = eval(models[2][model_num])
    test_model_ev = eval(models[3][model_num])
    
    score_list.append(evaluate_model(test_model, test_model_features, test_model_ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, verbose = True))

(4509, 13)

(4509,)

(4509, 2)

(250, 13)

(250,)

...

(248, 26)

(248,)

(248, 2)

,t1_odds,t2_odds,t1_prob,t2_prob,winner
0,130,-162,0.496339,0.503661,0
1,200,-265,0.516392,0.483608,1
2,-230,180,0.638861,0.361139,0
3,120,-150,0.520340,0.479660,1
4,100,-125,0.434664,0.565336,1
...,...,...,...,...,...
243,-157,137,0.569874,0.430126,0
244,-345,285,0.838180,0.161820,0
245,-185,160,0.624468,0.375532,1
246,-160,140,0.658574,0.341426,0



          Number of matches: 248
          Number of bets: 169
          Number of winning bets: 83
          Number of losing bets: 88
          Number of underdog bets: 118
          Number of underdog wins: 48
          Number of underdog losses: 70
          Number of Favorite bets: 50
          Number of favorite wins: 33
          Number of favorite losses: 17
          Number of even bets: 3
          Number of even wins: 2
          Number of even losses: 1
          Profit: 307.7349423225498
          Profit per bet: 1.820916818476626
          Profit per match: 1.2408667029135072
          
          


In [10]:
print(models[4])

['344.02261319277756', '345.1147018055343', '355.7009475540822', '-774.4056181289438', '1103.751826527694', '1770.9777157545009', '221.45128905823213', '1867.371333562617', '-200.6168769001822', '27.4555920635907', '245.96508405656152', '221.11659910561997', '690.5773086261729', '339.66093040809983', '-54.76505767745016']


In [11]:
print(score_list)

[832.9544224162701, 345.1147018055343, 355.7009475540822, -774.4056181289438, 1103.751826527694, 1961.130849366221, 221.45128905823213, 1867.371333562617, -300.61687690018186, -12.544407936409357, 245.96508405656152, 221.11659910561997, 640.5773086261727, 217.28330803047766, 307.7349423225498]


In [12]:
models[4] = score_list

In [13]:
models

[['model_0',
  'model_1',
  'model_2',
  'model_3',
  'model_4',
  'model_06_13_20',
  'model_06_20_20',
  'model_6_27_20',
  'model_7_18_20',
  'model_7_25_20',
  'model_9_2_20',
  'model_9_10_20',
  'model_10_23_20',
  'model_10_27_20',
  'model_10_28_20'],
 ['LogisticRegression(max_iter=50000, random_state=75)',
  "DecisionTreeClassifier(criterion='entropy', max_depth=10, max_leaf_nodes=336,\n                       min_samples_leaf=0.01, random_state=75,\n                       splitter='random')",
  "RandomForestClassifier(criterion='entropy', max_depth=83, max_leaf_nodes=20,\n                       min_samples_leaf=0.009999000000000001, n_estimators=39,\n                       n_jobs=-1, random_state=75)",
  "GradientBoostingClassifier(max_depth=17, max_features='sqrt',\n                           max_leaf_nodes=316, min_samples_leaf=0.1,\n                           n_estimators=195, random_state=75)",
  'GaussianNB()',
  'LogisticRegression(max_iter=50000, random_state=75, tol=5.

In [14]:
with open('../data/models.csv', 'w' , newline='') as outfile:
    writer = csv.writer(outfile)
    for row in models:
        print("HI")
        writer.writerow(row)
    
outfile.close()

HI
HI
HI
HI
HI
